# FBCSP + LDA (intra-sujeto y cross-sujeto)

### Bloque 1 — Rutas de salida + utilidades de logging y guardado

Qué hace: define las carpetas donde se guardarán figuras, tablas y logs bajo models/fbcsp_lda/. Incluye utilidades para inicializar un logger limpio, guardar matrices de confusión (PNG + CSV) y anexar métricas a CSVs acumulativos.

In [1]:
# %% [PATHS & LOGGING] — rutas de salida + helpers para logs/figuras/tablas
import sys, logging, warnings
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay
import mne

# Raíz del repo (este notebook está en models/fbcsp_lda/)
PROJ = Path('..').resolve().parent
DATA_PROC = PROJ / 'data' / 'processed'
OUT_ROOT = PROJ / 'models' / 'fbcsp_lda'
FIG_DIR  = OUT_ROOT / 'figures'
TAB_DIR  = OUT_ROOT / 'tables'
LOG_DIR  = OUT_ROOT / 'logs'
for d in (FIG_DIR, TAB_DIR, LOG_DIR):
    d.mkdir(parents=True, exist_ok=True)

print(f"Directorio de datos procesados: {DATA_PROC}")


Directorio de datos procesados: /root/Proyecto/EEG_Clasificador/data/processed


### Bloque 2 — FBCSP Helpers

Qué hace: helpers de modelo. Extraen X/y desde Epochs, aplican Filter-Bank + CSP por bandas y entrenan/escala LDA. Mantiene tus comentarios y añade compatibilidad con versiones de CSP.

In [2]:
# %% [HELPERS — comunes FBCSP/LOSO/Calibración/Logging]
# Reúne en un solo bloque:
#  - Descubrimiento de sujetos + DROP-only
#  - Perillas (_knobs_dict)
#  - FBCSP helpers (banco de filtros, picks motores, z-score por época, FBCSP transform)
#  - Clasificador (scaler + LDA)
#  - Split de calibración
#  - Logger

# ====== IMPORTS ======
import sys, logging, warnings, re
from glob import glob
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import mne

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from mne.decoding import CSP

# ====== RUTAS y ARCHIVOS AUXILIARES ======
# NOTA: este bloque asume que en un bloque anterior ya definiste:
#   PROJ, DATA_PROC, LOG_DIR  (del bloque PATHS & LOGGING original)
# Archivo opcional con sujetos a excluir (DROP-only)
STRICT_DROP_TXT = PROJ / 'reports' / 'tables' / '02_prepro' / 'subjects_strict_DROP.txt'
_re_sid = re.compile(r'^S\d{3}$')
# ====== FBCSP HELPERS ======
# Bancos de filtros
FB_BANDS_DENSE = [(f, f+2) for f in range(8, 30, 2)]                  # denso 8–30 por pasos de 2 Hz
FB_BANDS_CLASSIC = [(8,12), (12,16), (16,20), (20,24), (24,28), (28,30)]
DEFAULT_FB_BANDS = FB_BANDS_DENSE

# Nº de componentes CSP por sub-banda (típico 6–8 en bancos densos)
DEFAULT_N_CSP = 6

# LDA con shrinkage automático robusto
LDA_PARAMS = dict(solver='lsqr', shrinkage='auto')

# Picks motores (si deseas limitar; tus datos ya tienen 8 canales motores)
MOTOR_TOKENS = ['C3','C4','Cz','CP3','CP4','FC3','FC4','FCz']

def _list_subject_fifs(fif_dir=DATA_PROC, pattern='S???_MI-epo.fif'):
    """Devuelve lista ordenada de rutas a los FIF de sujetos disponibles."""
    return sorted(glob(str(fif_dir / pattern)))

def _list_available_subjects(fif_dir=DATA_PROC):
    """IDs únicos SXXX disponibles en el directorio de FIF procesados."""
    files = _list_subject_fifs(fif_dir)
    return sorted({Path(f).stem.split('_')[0] for f in files})

def _read_drop_file(path: Path):
    """Lee archivo con IDs de sujetos a excluir (uno por línea, formato SXXX)."""
    if not path.exists():
        return set()
    s = set()
    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        for ln in f:
            sid = ln.strip().upper()
            if _re_sid.match(sid):
                s.add(sid)
    return s

def _strict_valid_from_drop(avail_ids):
    """
    Aplica DROP-only:
      - Lee STRICT_DROP_TXT si existe
      - Devuelve (lista válidos, info string)
    """
    drop = _read_drop_file(STRICT_DROP_TXT)
    avail = set(avail_ids)
    valid = sorted(avail - drop)
    info = f"DROP-only: {len(drop)} en DROP; válidos={len(valid)}/{len(avail)}"
    if not STRICT_DROP_TXT.exists():
        info += " (archivo DROP no encontrado → sin exclusiones)"
    return valid, info

# Helper para registrar “perillas” (config) en logs/CSV
def _knobs_dict(crop_window, motor_only, zscore_epoch, fb_bands, n_csp):
    return dict(
        crop_window=crop_window if crop_window is not None else None,
        motor_only=bool(motor_only),
        zscore_epoch=bool(zscore_epoch),
        fb_bands=str(fb_bands),
        n_csp=int(n_csp)
    )

def _epochs_to_Xy(epochs: mne.Epochs):
    """
    Extrae X e y (clases string) desde Epochs respetando event_id.
    X: (n_epochs, n_channels, n_times)
    y: (n_epochs,) etiquetas string según mapping event_id.
    """
    X = epochs.get_data()
    inv = {v: k for k, v in epochs.event_id.items()}  # int->clase
    y = np.array([inv[e[-1]] for e in epochs.events], dtype=object)
    return X, y

def _find_motor_chs(ch_names, tokens=MOTOR_TOKENS):
    """Devuelve índices de canales que contienen tokens motores (case-insensitive)."""
    up = [c.upper() for c in ch_names]
    picks = []
    for tok in tokens:
        TU = tok.upper()
        for i, name in enumerate(up):
            if TU in name:
                picks.append(i); break
    return sorted(set(picks))

def _epochwise_zscore(X, eps=1e-8):
    """
    Z-score por época y canal (normaliza a lo largo del tiempo).
    X: (n_epochs, n_channels, n_times) → mismo shape.
    """
    mean = X.mean(axis=-1, keepdims=True)
    std  = X.std(axis=-1, keepdims=True)
    return (X - mean) / (std + eps)

def _fit_fb_csp_transform(train_ep: mne.Epochs,
                          test_ep: mne.Epochs,
                          fb_bands=DEFAULT_FB_BANDS,
                          n_csp=DEFAULT_N_CSP,
                          motor_only=False,
                          zscore_epoch=False,
                          crop_window=None):
    """
    Aplica FBCSP con opciones:
      - crop_window=(tmin,tmax): recorta épocas
      - motor_only=True: usa solo canales motores comunes
      - zscore_epoch=True: z-score por época/canal antes de CSP
      - fb_bands: lista de sub-bandas [(fmin,fmax), ...]
    Devuelve (Xtr_fb, Xte_fb) con features concatenadas por sub-banda.
    """
    tr = train_ep.copy()
    te = test_ep.copy()

    if crop_window is not None:
        tmin, tmax = crop_window
        tr.crop(tmin, tmax)
        te.crop(tmin, tmax)

    if motor_only:
        picks = _find_motor_chs(tr.ch_names)
        if picks:
            tr.pick(picks)
            # Alinear canales del test con los del train
            te = te.copy().reorder_channels(tr.ch_names)

    Xtr_list, Xte_list = [], []
    y_tr = tr.events[:, -1]

    for (fmin, fmax) in fb_bands:
        tr_b = tr.copy().filter(fmin, fmax, picks='eeg', verbose=False)
        te_b = te.copy().filter(fmin, fmax, picks='eeg', verbose=False)

        Xtr = tr_b.get_data()
        Xte = te_b.get_data()

        if zscore_epoch:
            Xtr = _epochwise_zscore(Xtr)
            Xte = _epochwise_zscore(Xte)

        try:
            csp = CSP(n_components=n_csp, reg='ledoit_wolf', log=True, norm_trace=False)
        except TypeError:
            # Compatibilidad con MNE antiguos sin 'norm_trace'
            csp = CSP(n_components=n_csp, reg='ledoit_wolf', log=True)

        Xtr_c = csp.fit_transform(Xtr, y_tr)
        Xte_c = csp.transform(Xte)

        Xtr_list.append(Xtr_c)
        Xte_list.append(Xte_c)

    Xtr_fb = np.concatenate(Xtr_list, axis=1)
    Xte_fb = np.concatenate(Xte_list,  axis=1)
    return Xtr_fb, Xte_fb

def _fit_scale_lda(Xtr, ytr, Xte, lda_params=LDA_PARAMS):
    """
    Estandariza features (fit solo en train) y entrena LDA.
    Devuelve: (yhat, clf, scaler)
    """
    scaler = StandardScaler()
    Xtr_s = scaler.fit_transform(Xtr)
    Xte_s = scaler.transform(Xte)

    clf = LDA(**lda_params)
    clf.fit(Xtr_s, ytr)
    yhat = clf.predict(Xte_s)
    return yhat, clf, scaler

# ====== CALIBRACIÓN  ======
def _split_calibration(ep_te, k_per_class=5, shuffle=True, random_state=42, 
                       require_all_classes=False, return_indices=False):
    """
    Divide un conjunto de épocas de TEST en:
      - CALIB (k_per_class por clase)
      - EVAL (el resto)

    Parámetros
    ----------
    ep_te : mne.Epochs
        Épocas del sujeto a partir de las cuales se hará calibración+evaluación.
    k_per_class : int
        Nº de épocas por clase que se irán a CALIB. Si <=0 → (None, ep_te).
    shuffle : bool
        Si True, baraja índices dentro de cada clase antes de tomar k.
        (Recomendado para evitar sesgo por orden temporal).
    random_state : int
        Semilla para la aleatoriedad (si shuffle=True).
    require_all_classes : bool
        Si True, exige que TODAS las clases tengan al menos k épocas;
        si alguna no alcanza, devuelve (None, ep_te).
        Si False, usa min(k, n_clase) y continúa.
    return_indices : bool
        Si True, además devuelve (idx_calib, idx_eval, class_counts).

    Retorna
    -------
    ep_calib : mne.Epochs or None
    ep_eval  : mne.Epochs
    (opcionales)
    idx_calib : np.ndarray (int)
    idx_eval  : np.ndarray (int)
    class_counts : dict {code: (n_calib, n_eval, n_total)}
    """
    if k_per_class <= 0:
        if return_indices:
            n = len(ep_te)
            idx_all = np.arange(n)
            labels = ep_te.events[:, -1]
            counts = {int(c): (0, int((labels == c).sum()), int((labels == c).sum()))
                      for c in np.unique(labels)}
            return None, ep_te, np.array([], dtype=int), idx_all, counts
        return None, ep_te

    labels = ep_te.events[:, -1].astype(int)
    classes = np.unique(labels)
    rng = np.random.RandomState(random_state) if shuffle else None

    calib_idx = []
    eval_idx  = []
    class_counts = {}

    # Chequeo opcional: todas las clases deben tener >= k
    if require_all_classes:
        for c in classes:
            n_c = int((labels == c).sum())
            if n_c < k_per_class:
                # no cumple el mínimo → no calibrar este sujeto
                if return_indices:
                    counts = {int(code): (0, int((labels == code).sum()), int((labels == code).sum()))
                              for code in classes}
                    return None, ep_te, np.array([], dtype=int), np.arange(len(ep_te)), counts
                return None, ep_te

    for c in classes:
        idx_c = np.where(labels == c)[0]
        if shuffle and len(idx_c) > 1:
            rng.shuffle(idx_c)

        take = min(k_per_class, len(idx_c))
        sel = idx_c[:take]
        rem = idx_c[take:]

        if take > 0:
            calib_idx.append(sel)
        if len(rem) > 0:
            eval_idx.append(rem)

        class_counts[int(c)] = (int(take), int(len(rem)), int(len(idx_c)))

    # Si no hay nada para calibrar, devolver (None, ep_te)
    if len(calib_idx) == 0:
        if return_indices:
            idx_all = np.arange(len(ep_te))
            return None, ep_te, np.array([], dtype=int), idx_all, class_counts
        return None, ep_te

    calib_idx = np.concatenate(calib_idx) if len(calib_idx) else np.array([], dtype=int)
    eval_idx  = np.concatenate(eval_idx)  if len(eval_idx)  else np.array([], dtype=int)

    # Ordenar índices para que cada subconjunto quede en orden cronológico
    calib_idx.sort()
    eval_idx.sort()

    ep_calib = ep_te.copy()[calib_idx]
    ep_eval  = ep_te.copy()[eval_idx] if len(eval_idx) > 0 else ep_te.copy()[[]]  # vacío si no hay eval

    if return_indices:
        return ep_calib, ep_eval, calib_idx, eval_idx, class_counts
    return ep_calib, ep_eval


# ====== LOGGER ======
def _init_logger(run_name: str):
    """
    Crea un logger que escribe a consola y a TXT en models/fbcsp_lda/logs/.
    Reduce la verbosidad de MNE para que los logs sean legibles.
    """
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    log_path = LOG_DIR / f"{ts}_{run_name}.txt"

    logger = logging.getLogger(run_name)
    logger.setLevel(logging.INFO)
    logger.handlers.clear()

    fmt = logging.Formatter("[%(asctime)s] %(levelname)s: %(message)s", datefmt="%H:%M:%S")
    ch = logging.StreamHandler(stream=sys.stdout); ch.setLevel(logging.INFO); ch.setFormatter(fmt)
    fh = logging.FileHandler(log_path, encoding="utf-8"); fh.setLevel(logging.INFO); fh.setFormatter(fmt)
    logger.addHandler(ch); logger.addHandler(fh)

    # Silenciar ruido externo (sin afectar tus prints/logs)
    mne.set_log_level("ERROR")
    warnings.filterwarnings("ignore", category=UserWarning, module="mne")
    warnings.filterwarnings("ignore", category=RuntimeWarning, module="mne")
    return logger, log_path


### Bloque 5 — Cross-sujeto (LOSO) con logs + guardado

Qué hace: para cada sujeto como test, entrena en el resto, calcula métricas y guarda una matriz de confusión por sujeto y una global. También guarda métricas por sujeto en tables/metrics_loso_per_subject.csv y un resumen global en tables/metrics_loso.csv, además de un TXT en logs/

- run_loso(..., use_strict=True) hace LOSO clásico sobre el conjunto resuelto (con Strict si está ON).

- run_loso_single(test_subject, ...) entrena con todos los demás y prueba sólo en ese sujeto (Strict opcional).

- Incluye utilidades de selección Strict y reemplazo para subject_list.

In [3]:
# %% [INTER-SUBJECT CV desde JSON de folds — NO usa archivo DROP; usa todos los sujetos del JSON]
import json
import numpy as np
import pandas as pd
import mne
from math import ceil
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,   # <<< NUEVO
)
import matplotlib.pyplot as plt
from pathlib import Path

def run_inter_subject_cv_from_json(
    fif_dir=DATA_PROC,
    folds_json_path=None,
    crop_window=(0.5, 3.5),
    motor_only=True,
    zscore_epoch=True,
    fb_bands=DEFAULT_FB_BANDS,
    n_csp=DEFAULT_N_CSP,
    calibrate_n=None,                  # calibración opcional con sujetos de test (modo antiguo)
    calibrate_k_per_class=None,        # <<< NUEVO: calibración per-subject con k épocas por clase
    val_ratio_subjects=0.16,           # % sujetos de TRAIN a VALID (≈ 13/82 ≈ 0.16)
    random_state=42,                   # reproducibilidad del split por sujeto
    max_subplots_per_fig=12,
    n_cols=4,
    save_csv_name=None,
    save_txt_name=None
):
    """
    Inter-subject CV usando folds del JSON, con VALIDACIÓN INTERNA POR SUJETOS.

    Modos de calibración:
      - Sin calibración: calibrate_n=None y calibrate_k_per_class=None o <=0.
      - Calibración por sujetos completos (modo antiguo): calibrate_n>0 → usa los primeros n sujetos de test para recalibrar, evalúa en el resto.
      - Calibración per-subject por k épocas/clase (RECOMENDADO): calibrate_k_per_class>0 → para cada sujeto de test, toma k épocas por clase
        para calibrar (junto con TRAIN) y evalúa en el resto de épocas DEL MISMO sujeto. Ignora calibrate_n.

    Antileakage:
      - FBCSP/Scaler/LDA se ajustan SOLO con datos de TRAIN (y, si hay calibración, con las muestras de calibración permitidas).
      - VAL y TEST se transforman con los modelos ajustados.
    """
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_tag = f"inter_subject_cv_json_{ts}"
    logger, log_path = _init_logger(run_name=run_tag)
    knobs = _knobs_dict(crop_window, motor_only, zscore_epoch, fb_bands, n_csp)

    # Normalizar flags de calibración
    k_cal = (0 if calibrate_k_per_class is None else int(calibrate_k_per_class))
    subj_cal = (0 if calibrate_n is None else int(calibrate_n))
    if k_cal > 0:
        # Modo per-subject (k-shots) tiene prioridad
        subj_cal = 0
        logger.info(f"[RUN {run_tag}] Inter-Subject CV con VALID por sujetos y CALIBRACIÓN PER-SUBJECT (k={k_cal} por clase)")
    elif subj_cal > 0:
        logger.info(f"[RUN {run_tag}] Inter-Subject CV con VALID por sujetos y CALIBRACIÓN por sujetos completos (n={subj_cal})")
    else:
        logger.info(f"[RUN {run_tag}] Inter-Subject CV con VALID por sujetos (SIN calibración)")
    logger.info(f"Perillas: {knobs} | val_ratio_subjects={val_ratio_subjects:.2f}")

    # JSON de folds
    if folds_json_path is None:
        folds_json_path = PROJ / 'models' / 'folds' / 'Kfold5.json'
    folds_json_path = Path(folds_json_path)
    if not folds_json_path.exists():
        raise FileNotFoundError(f"No se encontró folds JSON en {folds_json_path}")

    with open(folds_json_path, "r", encoding="utf-8") as f:
        payload = json.load(f)

    folds = payload.get("folds", [])
    subject_ids_json = payload.get("subject_ids", [])
    logger.info(f"Folds cargadas: {len(folds)} | sujetos en JSON: {len(subject_ids_json)}")

    # Cargar epochs por sujeto
    ep_map = {}
    for sid in subject_ids_json:
        fif_path = Path(fif_dir) / f"{sid}_MI-epo.fif"
        if fif_path.exists():
            try:
                ep_map[sid] = mne.read_epochs(str(fif_path), preload=True, verbose=False)
            except Exception as e:
                logger.warning(f"Error leyendo {fif_path} para {sid}: {e}")
        else:
            logger.warning(f"Falta archivo FIF para {sid}: {fif_path}")

    # Resultados
    rows = []
    cm_items = []
    cm_global = None
    classes_global = None
    per_fold_reports = []  # guardamos classification_report (TEST) por fold (y por sujeto si k-shots)

    for f in folds:
        fold_i = int(f.get("fold"))
        train_sids = [sid for sid in f.get("train", []) if sid in ep_map]
        test_sids  = [sid for sid in f.get("test", [])  if sid in ep_map]

        logger.info(f"[Fold {fold_i}] train({len(train_sids)}): {train_sids}")
        logger.info(f"[Fold {fold_i}] test ({len(test_sids)}): {test_sids}")

        if len(train_sids) == 0 or len(test_sids) == 0:
            logger.warning(f"[Fold {fold_i}] faltan sujetos train/test — saltando fold.")
            continue

        # ---------- VALIDACIÓN INTERNA POR SUJETOS ----------
        rng = np.random.RandomState(random_state + fold_i)
        n_val_subj = max(1, int(round(len(train_sids) * float(val_ratio_subjects))))
        val_indices = rng.choice(len(train_sids), size=n_val_subj, replace=False)
        val_sids = sorted([train_sids[i] for i in val_indices])
        tr_sids  = sorted([sid for sid in train_sids if sid not in set(val_sids)])

        logger.info(f"[Fold {fold_i}] split interno → train_sids={len(tr_sids)}, val_sids={len(val_sids)}")

        # Concatenar epochs por split
        ep_tr  = mne.concatenate_epochs([ep_map[sid] for sid in tr_sids],  on_mismatch='ignore')
        ep_val = mne.concatenate_epochs([ep_map[sid] for sid in val_sids], on_mismatch='ignore')
        ep_te  = mne.concatenate_epochs([ep_map[sid] for sid in test_sids], on_mismatch='ignore')

        # Alinear canales
        try:
            ep_val = ep_val.copy().reorder_channels(ep_tr.ch_names)
            ep_te  = ep_te.copy().reorder_channels(ep_tr.ch_names)
        except Exception as e:
            logger.warning(f"[Fold {fold_i}] reorder_channels: {e}")

        # Etiquetas
        _, y_tr_str  = _epochs_to_Xy(ep_tr)
        _, y_val_str = _epochs_to_Xy(ep_val)
        _, y_te_str  = _epochs_to_Xy(ep_te)

        # (purista) podrías usar solo train+val; manteneremos train+val+test para robustez de mapeo de clases
        le = LabelEncoder().fit(np.concatenate([y_tr_str, y_val_str, y_te_str]))
        y_tr  = le.transform(y_tr_str)
        y_val = le.transform(y_val_str)
        y_te  = le.transform(y_te_str)
        classes = list(le.classes_)

        if classes_global is None:
            classes_global = classes
            cm_global = np.zeros((len(classes), len(classes)), dtype=int)

        # ---------- FEATURES FBCSP (ajuste SOLO con TRAIN-SUBJECTS) ----------
        with mne.utils.use_log_level("ERROR"):
            # Fit en ep_tr, transform en ep_val y ep_te
            Xtr_fb, Xval_fb = _fit_fb_csp_transform(
                ep_tr, ep_val,
                fb_bands=fb_bands,
                n_csp=n_csp,
                motor_only=motor_only,
                zscore_epoch=zscore_epoch,
                crop_window=crop_window
            )
            # Xte con ajuste de ep_tr (solo para el caso sin calibración)
            _, Xte_fb = _fit_fb_csp_transform(
                ep_tr, ep_te,
                fb_bands=fb_bands,
                n_csp=n_csp,
                motor_only=motor_only,
                zscore_epoch=zscore_epoch,
                crop_window=crop_window
            )

        # ---------- ENTRENAR CLASIFICADOR SOLO CON TRAIN ----------
        yhat_val, clf_val, scaler_val = _fit_scale_lda(Xtr_fb, y_tr, Xval_fb)
        acc_val = accuracy_score(y_val, yhat_val)
        f1m_val = f1_score(y_val, yhat_val, average='macro')
        logger.info(f"[Fold {fold_i}] VAL   acc={acc_val:.4f} | f1m={f1m_val:.4f} | n_val={len(y_val)}")

        # --------- TEST: tres caminos ----------
        if k_cal > 0:
            # ===== Calibración per-subject por k épocas/clase =====
            y_te_all, yhat_all = [], []
            cm_fold = np.zeros((len(classes), len(classes)), dtype=int)

            for sid in test_sids:
                # Reordenar canales del sujeto de test contra TRAIN
                ep_te_subj = ep_map[sid].copy()
                try:
                    ep_te_subj = ep_te_subj.reorder_channels(ep_tr.ch_names)
                except Exception as e:
                    logger.warning(f"[Fold {fold_i}] reorder (test subject {sid}): {e}")

                # Partir en CALIB vs EVAL (k por clase)
                ep_calib, ep_eval = _split_calibration(ep_te_subj, k_per_class=k_cal)
                if (ep_calib is None) or (len(ep_calib) == 0) or (len(ep_eval) == 0):
                    logger.warning(f"[Fold {fold_i}] {sid}: calibración insuficiente (k={k_cal}) o sin muestras de eval; se omite este sujeto.")
                    continue

                # Etiquetas
                _, y_calib_str = _epochs_to_Xy(ep_calib)
                _, y_eval_str  = _epochs_to_Xy(ep_eval)
                y_calib = le.transform(y_calib_str)
                y_eval  = le.transform(y_eval_str)

                # Refit features con TRAIN + CALIB_de_este_sujeto; transformar EVAL
                with mne.utils.use_log_level("ERROR"):
                    ep_train_plus_calib = mne.concatenate_epochs([ep_tr, ep_calib], on_mismatch='ignore')
                    Xtr_comb, Xeval = _fit_fb_csp_transform(
                        ep_train_plus_calib, ep_eval,
                        fb_bands=fb_bands,
                        n_csp=n_csp,
                        motor_only=motor_only,
                        zscore_epoch=zscore_epoch,
                        crop_window=crop_window
                    )

                y_tr_comb = np.concatenate([y_tr, y_calib])
                yhat_eval, _, _ = _fit_scale_lda(Xtr_comb, y_tr_comb, Xeval)

                # Acumular por sujeto
                y_te_all.append(y_eval)
                yhat_all.append(yhat_eval)

                cm_s = confusion_matrix(y_eval, yhat_eval, labels=np.arange(len(classes)))
                cm_fold += cm_s
                logger.info(f"[Fold {fold_i}] TEST (per-subject k-shots) {sid} → acc={accuracy_score(y_eval, yhat_eval):.4f}, n={len(y_eval)}")

            if (len(y_te_all) == 0):
                logger.warning(f"[Fold {fold_i}] Sin sujetos válidos para calibración per-subject (k={k_cal}). Se usa modelo sin calibrar.")
                yhat_te = clf_val.predict(scaler_val.transform(Xte_fb))
                y_te_cat = y_te
                cm = confusion_matrix(y_te_cat, yhat_te, labels=np.arange(len(classes)))
            else:
                y_te_cat  = np.concatenate(y_te_all)
                yhat_te   = np.concatenate(yhat_all)
                cm = cm_fold

        elif subj_cal > 0:
            # ===== Calibración por sujetos completos (modo antiguo) =====
            n_subjs = min(int(subj_cal), len(test_sids))
            if n_subjs >= len(test_sids):
                logger.warning(f"[Fold {fold_i}] calibrate_n ({subj_cal}) >= nº test_sids ({len(test_sids)}). Se reducirá a {len(test_sids)-1}.")
                n_subjs = max(0, len(test_sids) - 1)

            calib_sids = test_sids[:n_subjs]
            rest_sids  = test_sids[n_subjs:]
            ep_calib = mne.concatenate_epochs([ep_map[sid] for sid in calib_sids], on_mismatch='ignore') if calib_sids else None
            ep_te_rest = mne.concatenate_epochs([ep_map[sid] for sid in rest_sids], on_mismatch='ignore') if rest_sids else None

            if ep_calib is None or ep_te_rest is None:
                # sin calibración efectiva o sin test restante
                yhat_te = clf_val.predict(scaler_val.transform(Xte_fb))
                y_te_cat = y_te
            else:
                try:
                    ep_calib   = ep_calib.copy().reorder_channels(ep_tr.ch_names)
                    ep_te_rest = ep_te_rest.copy().reorder_channels(ep_tr.ch_names)
                except Exception as e:
                    logger.warning(f"[Fold {fold_i}] reorder (calib/test_rest): {e}")

                _, y_calib_str = _epochs_to_Xy(ep_calib)
                y_calib = le.transform(y_calib_str)

                with mne.utils.use_log_level("ERROR"):
                    ep_train_plus_calib = mne.concatenate_epochs([ep_tr, ep_calib], on_mismatch='ignore')
                    Xtr_comb, Xte_rest = _fit_fb_csp_transform(
                        ep_train_plus_calib, ep_te_rest,
                        fb_bands=fb_bands,
                        n_csp=n_csp,
                        motor_only=motor_only,
                        zscore_epoch=zscore_epoch,
                        crop_window=crop_window
                    )

                y_tr_comb = np.concatenate([y_tr, y_calib])
                yhat_te = _fit_scale_lda(Xtr_comb, y_tr_comb, Xte_rest)[0]
                _, y_te_rest_str = _epochs_to_Xy(ep_te_rest)
                y_te_cat = le.transform(y_te_rest_str)

            cm = confusion_matrix(y_te_cat, yhat_te, labels=np.arange(len(classes)))

        else:
            # ===== Sin calibración =====
            y_te_cat = y_te
            yhat_te = clf_val.predict(scaler_val.transform(Xte_fb))
            cm = confusion_matrix(y_te_cat, yhat_te, labels=np.arange(len(classes)))

        # ---------- MÉTRICAS TEST ----------
        acc = accuracy_score(y_te_cat, yhat_te)
        f1m = f1_score(y_te_cat, yhat_te, average='macro')
        cm_global += cm

        # Reports
        cls_rep = classification_report(y_te_cat, yhat_te, target_names=classes, digits=4)
        per_fold_reports.append((fold_i, cls_rep))
        logger.info(f"[Fold {fold_i}] TEST  acc={acc:.4f} | f1m={f1m:.4f} | n_test={len(y_te_cat)}")
        print(f"[Fold {fold_i}] Classification report (TEST):\n{cls_rep}")

        rows.append(dict(
            fold=int(fold_i),
            train_subjects=",".join(tr_sids),
            val_subjects=",".join(val_sids),
            test_subjects=",".join(test_sids),
            val_acc=float(acc_val),
            val_f1_macro=float(f1m_val),
            acc=float(acc),
            f1_macro=float(f1m),
            n_val=int(len(y_val)),
            n_test=int(len(y_te_cat)),
            calibrate_mode=("k-per-class" if k_cal > 0 else ("subjects" if subj_cal > 0 else "none")),
            calibrate_param=(k_cal if k_cal > 0 else (subj_cal if subj_cal > 0 else 0))
        ))
        cm_items.append((f"fold_{fold_i}", cm, classes))

    # ---------- Consolidados ----------
    df_rows = pd.DataFrame(rows).sort_values("fold") if rows else pd.DataFrame()
    acc_mu   = float(df_rows['acc'].mean()) if not df_rows.empty else 0.0
    f1_mu    = float(df_rows['f1_macro'].mean()) if not df_rows.empty else 0.0
    val_mu   = float(df_rows['val_acc'].mean()) if not df_rows.empty else 0.0
    valf1_mu = float(df_rows['val_f1_macro'].mean()) if not df_rows.empty else 0.0

    if not df_rows.empty:
        df_rows = pd.concat([df_rows, pd.DataFrame([{
            'fold': 0,
            'train_subjects': 'GLOBAL',
            'val_subjects': 'GLOBAL',
            'test_subjects': 'GLOBAL',
            'val_acc': val_mu,
            'val_f1_macro': valf1_mu,
            'acc': acc_mu,
            'f1_macro': f1_mu,
            'n_val': int(df_rows['n_val'].sum()),
            'n_test': int(df_rows['n_test'].sum()),
            'calibrate_mode': df_rows['calibrate_mode'].mode()[0] if 'calibrate_mode' in df_rows else 'none',
            'calibrate_param': int(df_rows['calibrate_param'].mean()) if 'calibrate_param' in df_rows else 0
        }])], ignore_index=True)

    out_csv = (TAB_DIR / f"{ts}_{save_csv_name}") if save_csv_name else (TAB_DIR / f"metrics_inter_subject_cv_{ts}.csv")
    df_rows.to_csv(out_csv, index=False)
    logger.info(f"CSV consolidado → {out_csv}")
    print("CSV consolidado →", out_csv)

    out_txt = (LOG_DIR / f"{ts}_{save_txt_name}") if save_txt_name else (LOG_DIR / f"metrics_inter_subject_cv_{ts}.txt")
    with open(out_txt, "w", encoding="utf-8") as f:
        f.write("INTER-SUBJECT CV (folds JSON) — Con VALID interno por sujetos\n")
        f.write(f"Generado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Total filas: {len(df_rows)}\n\n")
        if not df_rows.empty:
            header = df_rows.columns.tolist()
            f.write(" | ".join(header) + "\n")
            f.write("-" * 160 + "\n")
            for _, row in df_rows.iterrows():
                vals = []
                for kcol in header:
                    v = row[kcol]
                    if isinstance(v, float):
                        vals.append(f"{v:.4f}")
                    elif isinstance(v, (np.integer,)):
                        vals.append(str(int(v)))
                    else:
                        vals.append(str(v))
                f.write(" | ".join(vals) + "\n")
    logger.info(f"TXT consolidado → {out_txt}")
    print("TXT consolidado →", out_txt)

    # TXT con todos los classification reports por fold (TEST)
    reports_txt = LOG_DIR / f"classification_reports_by_fold_{ts}.txt"
    with open(reports_txt, "w", encoding="utf-8") as f:
        f.write("INTER-SUBJECT CV — Classification reports por fold (TEST)\n")
        f.write(f"Generado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        for fold_i, rep in sorted(per_fold_reports, key=lambda x: x[0]):
            f.write(f"[Fold {fold_i}] Classification report (TEST)\n")
            f.write(rep)
            if not rep.endswith("\n"):
                f.write("\n")
            f.write("-" * 80 + "\n")
    logger.info(f"TXT de classification reports → {reports_txt}")
    print("TXT de classification reports →", reports_txt)

    # Mosaicos de confusión por fold
    if cm_items:
        n = len(cm_items)
        per_fig = max(1, int(max_subplots_per_fig))
        n_figs = ceil(n / per_fig)
        n_rows_per_fig = lambda count: ceil(count / n_cols)
        for fig_idx in range(n_figs):
            start = fig_idx * per_fig
            end   = min((fig_idx + 1) * per_fig, n)
            chunk = cm_items[start:end]
            count = len(chunk)
            n_rows = n_rows_per_fig(count)

            fig, axes = plt.subplots(n_rows, n_cols, figsize=(4.5*n_cols, 3.8*n_rows), dpi=140)
            axes = np.atleast_2d(axes).flatten()
            for ax_i, (label, cm_sum, classes) in enumerate(chunk):
                ax = axes[ax_i]
                disp = ConfusionMatrixDisplay(cm_sum, display_labels=classes)
                disp.plot(ax=ax, cmap="Blues", colorbar=False, values_format='d')
                ax.set_title(f"{label}")
                ax.set_xlabel(""); ax.set_ylabel("")
            for j in range(ax_i + 1, len(axes)):
                axes[j].axis("off")

            out_png = FIG_DIR / f"inter_subject_confusions_{ts}_p{fig_idx+1}.png"
            fig.suptitle(f"Inter-Subject CV — Matrices de confusión (página {fig_idx+1}/{n_figs})", y=0.995, fontsize=14)
            fig.tight_layout(rect=[0, 0, 1, 0.97])
            fig.savefig(out_png)
            plt.close(fig)
            logger.info(f"Figura consolidada → {out_png}")
            print("Figura consolidada →", out_png)

    # Matriz GLOBAL
    if cm_global is not None and classes_global is not None:
        fig, ax = plt.subplots(figsize=(6.5, 5.2), dpi=140)
        disp = ConfusionMatrixDisplay(cm_global, display_labels=classes_global)
        disp.plot(ax=ax, cmap="Blues", colorbar=True, values_format='d')
        ax.set_title("Inter-Subject CV — Matriz de confusión GLOBAL (sumatoria folds)")
        fig.tight_layout()
        out_png_glob = FIG_DIR / f"inter_subject_global_confusion_{ts}.png"
        fig.savefig(out_png_glob)
        plt.close(fig)
        logger.info(f"Matriz GLOBAL → {out_png_glob}")
        print("Matriz GLOBAL →", out_png_glob)

    logger.info(f"[GLOBAL] VAL_acc={val_mu:.3f} | VAL_f1m={valf1_mu:.3f} | TEST_acc={acc_mu:.3f} | TEST_f1m={f1_mu:.3f}")
    print(f"[GLOBAL] VAL_acc={val_mu:.3f} | VAL_f1m={valf1_mu:.3f} | TEST_acc={acc_mu:.3f} | TEST_f1m={f1_mu:.3f}")
    logger.info(f"Log global de esta corrida → {log_path}")
    print(f"Log global → {log_path}")

    return df_rows.reset_index(drop=True)


### Bloque 7 — Ejemplos de ejecución

Qué hace: muestra cómo lanzar los “batch” por defecto (4 intra, 2 LOSO) y cómo pasar propia lista de sujetos.

In [4]:
# INTRA en todos los sujetos
# df_intra_optimal = run_intra_all(
#     k=5,
#     random_state=42,
#     crop_window=(0.5, 4.5),      # VENTANA ORIGINAL PROBADA
#     motor_only=True,
#     zscore_epoch=False,
#     fb_bands=FB_BANDS_DENSE,     # 11 BANDS ORIGINALES
#     n_csp=4,
#     save_txt_name="intra_optimal_proven.txt"
# )

# Inter-subject CV usando JSON de folds (reemplaza LOSO clásico)
# Ajusta folds_json_path si usaste otra ruta; por defecto busca PRO|        |J/models/folds/Kfold5.json
df_inter = run_inter_subject_cv_from_json(
    fif_dir=DATA_PROC,
    folds_json_path=PROJ / 'models' / '00_folds' / 'Kfold5.json',
    crop_window=(0.5, 3.5),
    motor_only=True,
    zscore_epoch=False,
    fb_bands=FB_BANDS_DENSE,
    n_csp=4,
    calibrate_k_per_class=5,   # ← activa calibración per-subject con 5 épocas/clase
    save_csv_name="inter_subject_cv_from_json.csv"
)


[04:05:10] INFO: [RUN inter_subject_cv_json_20251030-040510] Inter-Subject CV con VALID por sujetos y CALIBRACIÓN PER-SUBJECT (k=5 por clase)


INFO:inter_subject_cv_json_20251030-040510:[RUN inter_subject_cv_json_20251030-040510] Inter-Subject CV con VALID por sujetos y CALIBRACIÓN PER-SUBJECT (k=5 por clase)


[04:05:10] INFO: Perillas: {'crop_window': (0.5, 3.5), 'motor_only': True, 'zscore_epoch': False, 'fb_bands': '[(8, 10), (10, 12), (12, 14), (14, 16), (16, 18), (18, 20), (20, 22), (22, 24), (24, 26), (26, 28), (28, 30)]', 'n_csp': 4} | val_ratio_subjects=0.16


INFO:inter_subject_cv_json_20251030-040510:Perillas: {'crop_window': (0.5, 3.5), 'motor_only': True, 'zscore_epoch': False, 'fb_bands': '[(8, 10), (10, 12), (12, 14), (14, 16), (16, 18), (18, 20), (20, 22), (22, 24), (24, 26), (26, 28), (28, 30)]', 'n_csp': 4} | val_ratio_subjects=0.16


[04:05:10] INFO: Folds cargadas: 5 | sujetos en JSON: 103


...

Matriz GLOBAL → /root/Proyecto/EEG_Clasificador/models/fbcsp_lda/figures/inter_subject_global_confusion_20251030-040510.png
[06:00:07] INFO: [GLOBAL] VAL_acc=0.347 | VAL_f1m=0.345 | TEST_acc=0.365 | TEST_f1m=0.362


INFO:inter_subject_cv_json_20251030-040510:[GLOBAL] VAL_acc=0.347 | VAL_f1m=0.345 | TEST_acc=0.365 | TEST_f1m=0.362


[GLOBAL] VAL_acc=0.347 | VAL_f1m=0.345 | TEST_acc=0.365 | TEST_f1m=0.362
[06:00:07] INFO: Log global de esta corrida → /root/Proyecto/EEG_Clasificador/models/fbcsp_lda/logs/20251030-040510_inter_subject_cv_json_20251030-040510.txt


INFO:inter_subject_cv_json_20251030-040510:Log global de esta corrida → /root/Proyecto/EEG_Clasificador/models/fbcsp_lda/logs/20251030-040510_inter_subject_cv_json_20251030-040510.txt


Log global → /root/Proyecto/EEG_Clasificador/models/fbcsp_lda/logs/20251030-040510_inter_subject_cv_json_20251030-040510.txt
